# BI Exam May 2025: COVID-19 Data

#### Created by Group 7 - Kamilla, Jeanette, Juvena

In [ ]:
# import libraries
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import sklearn.metrics as sm
from scipy import stats
from sklearn import metrics
from sklearn import tree
from scipy.spatial.distance import cdist
from sklearn import preprocessing as prep
from sklearn.decomposition import PCA
from sklearn.preprocessing import StandardScaler, MinMaxScaler, MaxAbsScaler, QuantileTransformer
from sklearn.feature_selection import SelectKBest, f_classif
from sklearn.metrics import accuracy_score, classification_report
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestRegressor
from sklearn.cluster import KMeans
from sklearn.metrics import silhouette_score
from sklearn import model_selection
from sklearn.tree import DecisionTreeClassifier
from sklearn.linear_model import LinearRegression
from sklearn.metrics import r2_score
from sklearn.metrics import explained_variance_score

# Set plot styles for better visualization
plt.style.use('seaborn-v0_8-whitegrid')
sns.set_palette("Set2")

# Data Preparation

### 1. Load the Data

Now that we have our tools ready, the next step is to load the COVID-19 dataset into Python so we can start analyzing it.

In this case, we’re working with a single dataset:

- **OWID COVID-19 Latest Data**: a CSV file that contains country-level information on cases, deaths, vaccinations, testing, and various socioeconomic indicators.

We'll use Pandas to read the CSV file and store it as a DataFrame. To make our code cleaner and reusable, we'll define a simple function that loads the data and performs some initial checks. This way, we can easily reload or replace the dataset if needed in future steps.

In [ ]:
# File paths for the covid datasets. (dataset: last updated 2024-08-04)
dataset_covid = 'Data/owid-covid-latest.csv'

# Function to load the Excel files
def load_csv_to_dataframe(file_path):
    # Reads the Excel file and skips the first row if it contains a description or title
    df = pd.read_csv(file_path)
    return df

# Load datasets
print("..Loading COVID-19 dataset")
df_covid = load_csv_to_dataframe(dataset_covid)

---

### 2. Explore the Data

After loading the dataset, we want to explore it to understand what kind of information it contains and how it's structured.

To do this, we can use several helpful Pandas functions such as `shape`, `types`, `info()`, `head()`, `tail()`, `sample()`, `describe()` and `isnull().sum()`. These functions will give us insights into the number of rows and columns, the data types of each column, a summary of the data, and any missing values. 

This exploration is crucial as it helps us identify potential issues or areas that need further cleaning or transformation before we proceed with our analysis. 

In [ ]:
# Check the shape of the DataFrame (rows, columns)
df_covid.shape

In [ ]:
# Display the types of attributes (colum names) in the DataFrame
df_covid.dtypes

In [ ]:
# Gives an overview of the DataFrame
df_covid.info()

In [ ]:
# Display the first 5 rows of the DataFrame
df_covid.head()

In [ ]:
# Display the last 5 rows of the DataFrame
df_covid.tail()

In [ ]:
# Display a random sample of 5 rows from the DataFrame
df_covid.sample(5)

In [ ]:
# Gives summary statistics for all numerical columns in the dataset
df_covid.describe()

##### **2.1 Summary of exploring the data**

After exploring the dataframe, we found that it contains a large number of columns, many of which are not useful for our analysis or modeling goals. While some columns provide valuable information (like total cases, deaths, and vaccination rates), others are either redundant, mostly empty, or irrelevant.

This highlights the need for a thorough data cleaning step to remove unnecessary columns, handle missing values, and focus only on the most relevant features for our machine learning tasks.

---

### 3. Clean the Data

After loading and exploring the data, we need to clean it to ensure that our analysis is accurate and meaningful. Data cleaning involves several steps, including: checking for missing values, removing duplicates, and converting data types.

We start by doing a bit of cleaning of the big dataset, to remove rows and columns that are not relevant for our futher analysis and before we seperate the data into more specific datasets.

In [ ]:
# Check for missing values in the DataFrame
df_covid.isnull().sum()

The output above shows that many columns contain no values at all, so we will remove them to clean up the dataset.

In [ ]:
# Before cleaning the data, we want to remove irrelevant OWID aggregate rows—such as those representing high-income, low-income, and other income groupings.
rows_to_remove = ["OWID_UMC", "OWID_WRL", "OWID_LMC", "OWID_LIC", "OWID_HIC"]
df_removed_rows = df_covid[~df_covid["iso_code"].isin(rows_to_remove)]

We are removing the 'low-income countries', 'lower-middle-income countries', 'upper-middle-income countries', 'high-income countries' and 'world' categories because they are too broad and lack specific country-level detail, making it difficult to draw meaningful conclusions without relying on assumptions.

In [ ]:
# Checking if the above rows were removed
print(f"{df_covid.shape}")
print(f"Removed the {df_covid.shape[0] - df_removed_rows.shape[0]} OWID rows from the dataframe.")

In [ ]:
# We will drop all columns with no values at all like; excess_mortality_cumulative_absolute, excess_mortality_cumulative etc.
df_covid_removed_columns = df_removed_rows.dropna(axis=1, how='all')

In [ ]:
# Check whether the columns were removed
print(f"COVID dataframe shape after removing columns: {df_covid_removed_columns.shape}")
print(f"Removed {df_covid.shape[1] - df_covid_removed_columns.shape[1]} columns from the dataframe.")

#### 3.1 Separating the data into different datasets

Now we separate the continent-level , age-level and health-level data into their own DataFrames so that we can clean and process them independently from the country-level data. This allows us to apply different cleaning steps based on the nature of the data, since data may have different structures or missing values compared to individual countries.

##### 3.1.1 Separating the continent-level data

In [ ]:
# Function to filter the DataFrame based on a list of values
def filter_dataframe(df, values, filter_type='rows', row_filter_column=None):
    if filter_type == 'rows':
        if row_filter_column is None:
            raise ValueError("Must specify 'row_filter_column' when filtering rows.")
        return df[df[row_filter_column].isin(values)]
    elif filter_type == 'columns':
        # Keep only columns present in df and in values list (avoid key error)
        columns_to_keep = [col for col in values if col in df.columns]
        return df[columns_to_keep]
    else:
        raise ValueError("filter_type must be either 'rows' or 'columns'")

In [ ]:
# Before removing the iso_code column, we want to secure the OWID fields for the continents since it could be relevant data to analyze.
rows_to_secure = ["OWID_AFR", "OWID_ASI", "OWID_EUR", "OWID_EUN", "OWID_NAM", "OWID_OCE", "OWID_SAM"]
df_continents = filter_dataframe(df_covid_removed_columns, rows_to_secure, filter_type='rows', row_filter_column='iso_code')

In [ ]:
# Check if the rows were secured
df_continents

We now have a new seperate dataframe called `df_continent` that contains the continent-level data. This DataFrame will be used for further analysis and modeling, while the original `df_covid` DataFrame will focus on country-level data.

In [ ]:
# Remove the columns that are irrelavnt since they contain no values at all like; population_density, median_age etc.
df_continents_romved_columns = df_continents.dropna(axis=1, how='all')
df_continents_romved_columns

The columns `new_vaccinations_smoothed_per_million`, `new_people_vaccinated_smoothed` and `new_people_vaccinated_smoothed_per_hundred` contain a lot of missing values so they are not necessary for our analysis and we will drop them from the `df_continents_cleaned` DataFrame. By removing them, we can simplify the DataFrame and focus on the most relevant features for our analysis.

In [ ]:
# Check whether the columns were removed
print(f"df_continent shape after removing columns: {df_continents_romved_columns.shape}")
print(f"Removed {df_continents.shape[1] - df_continents_romved_columns.shape[1]} columns from the dataframe.")

In [ ]:
# Removing columns there are irrelevant 
df_continents_cleaned = df_continents_romved_columns.drop(['iso_code', 'new_vaccinations_smoothed_per_million', 'new_people_vaccinated_smoothed', 'new_people_vaccinated_smoothed_per_hundred'], axis=1)
df_continents_cleaned

In [ ]:
# Check for duplicates in the DataFrame
df_continents_cleaned.duplicated().sum()

We still have some rows with missing values in the df_continents_cleaned DataFrame, so we will impute them to ensure that our analysis is accurate and meaningful. This step is important because missing values can lead to biased results or errors in our models.

Since it is a small dataframe, we can't delete the row with missing values(NaN), since we will lose a lot of data. We choose to replace the missing values, even though it can have a high risk of giving wrong information and have a big impact. 

The first four we replaced using realistic data.
But since it took too much time, we’ll use the median to fill the remaining NaN values.

In [ ]:
# method for replacing cell with a value
def replace_cell(df, row_filter, column, value):
    df.loc[row_filter, column] = value

# replace NaN for total_vaccinations for Africa. 1.084.500.000 is from Africa CDC, which is offical and reliable.
replace_cell(df_continents_cleaned, df_continents_cleaned['location'] == 'Africa', 'total_vaccinations', 1084500000)

# replace NaN for total_vaccinations for South America. 970.800.000 is from WTO-IMF COVID-19 Vaccine Trade Tracker, which is offical and reliable.
replace_cell(df_continents_cleaned, df_continents_cleaned['location'] == 'South America', 'total_vaccinations', 970800000)

# replace NaN for people_vaccinated for Africa. 725.000.000 is from Africa CDC, which is offical and reliable
replace_cell(df_continents_cleaned, df_continents_cleaned['location'] == 'Africa', 'people_vaccinated', 725000000)

# replace NaN for people_vaccinated for South America. 351.310.000 is from Our World in Data which is offical and reliable used by WHO.
replace_cell(df_continents_cleaned, df_continents_cleaned['location'] == 'South America', 'people_vaccinated', 351310000)

# method for replacing cell with median 
def fill_na_with_median(df, column_name):
    median_value = df[column_name].median()
    print(f"Median of '{column_name}': {median_value:.2f}")
    df[column_name].fillna(median_value, inplace=True)


fill_na_with_median(df_continents_cleaned, 'people_fully_vaccinated')
fill_na_with_median(df_continents_cleaned, 'total_boosters')
fill_na_with_median(df_continents_cleaned, 'new_vaccinations')
fill_na_with_median(df_continents_cleaned, 'new_vaccinations_smoothed')
fill_na_with_median(df_continents_cleaned, 'total_vaccinations_per_hundred')
fill_na_with_median(df_continents_cleaned, 'people_vaccinated_per_hundred')
fill_na_with_median(df_continents_cleaned, 'people_fully_vaccinated_per_hundred')
fill_na_with_median(df_continents_cleaned, 'total_boosters_per_hundred')
df_continents_cleaned

In [ ]:
# Check for missing values in the DataFrame
df_continents_cleaned.isnull().sum()

##### 3.1.2 Separating the age-level data

In [ ]:
# We are using the same function as above to seperate the age-level columns from the rest of the data.
columns_to_secure = ["continent", "location", "total_deaths_per_million", "median_age", "aged_65_older", "aged_70_older", "life_expectancy"]
df_age = filter_dataframe(df_covid_removed_columns, columns_to_secure, filter_type='columns')

In [ ]:
# Check if the rows were secured
df_age

We now have a new seperate dataframe called `df_age` that contains the age-level data. This DataFrame will be used for further analysis and modeling, while the original `df_covid` DataFrame will focus on country-level data.

In [ ]:
# Check for missing values in the DataFrame
df_age.isnull().sum()

In [ ]:
# Drop all the rows with NaN values in the 'median_age' column
df_age_cleaned = df_age.dropna(subset=['median_age'])

In [ ]:
# Do another check for missing values in the DataFrame
df_age_cleaned.isnull().sum()

There are still some missing values in the `df_age_cleaned` DataFrame, so we will impute them to ensure that our analysis is accurate and meaningful. This step is important because missing values can lead to biased results or errors in our models.

In [ ]:
# Fill NaN values with the median for the columns; total_deaths_per_million, aged_65_older and aged_70_older
fill_na_with_median(df_age_cleaned, "total_deaths_per_million")
fill_na_with_median(df_age_cleaned, "aged_65_older")
fill_na_with_median(df_age_cleaned, "aged_70_older")
df_age_cleaned

In [ ]:
# Check for duplicates in the DataFrame
df_age_cleaned.duplicated().sum()

##### 3.1.3 Separating the health-level data

In [ ]:
# We are using the same function as above to seperate the health-level columns from the rest of the data.
columns_to_secure = ["continent", "location", "total_deaths_per_million", "cardiovasc_death_rate", "diabetes_prevalence", "female_smokers", "male_smokers", "life_expectancy"]
df_health = filter_dataframe(df_covid_removed_columns, columns_to_secure, filter_type='columns')

In [ ]:
# Check if the rows were secured
df_health

We now have a new seperate dataframe called `df_health` that contains age-level data. This DataFrame will be used for further analysis and modeling, while the original `df_covid` DataFrame will focus on country-level data.

In [ ]:
# Check for missing values in the DataFrame
df_health.isnull().sum()

In [ ]:
# Drop all the rows with NaN values in the 'female_smokers' column
df_health_cleaned = df_health.dropna(subset=['female_smokers'])

In [ ]:
# Do another check for missing values in the DataFrame
df_health_cleaned.isnull().sum()

There are still some missing values in the `df_health_cleaned` DataFrame, so we will impute them to ensure that our analysis is accurate and meaningful. This step is important because missing values can lead to biased results or errors in our models.

In [ ]:
# Fill NaN values with the median for the columns; cardiovasc_death_rate and male_smokers
fill_na_with_median(df_health_cleaned, "cardiovasc_death_rate")
fill_na_with_median(df_health_cleaned, "male_smokers")
df_health_cleaned

In [ ]:
# Check for duplicates in the DataFrame
df_health_cleaned.duplicated().sum()

#### 3.2 Futher cleaning of the country-level data 

We have selected a subset of columns that we consider relevant for our analysis. This subset includes columns that provide information on total cases, deaths and population. By focusing on these columns, we can simplify our analysis and make it easier to draw meaningful conclusions.

In [ ]:
# We make a new dataframe with the columns we want to keep for future analysis.
columns_we_want_to_keep = [
    "iso_code", "continent", "location", "total_cases", "total_deaths",
    "total_cases_per_million", "total_deaths_per_million",
    "life_expectancy", "population"]

# Removes all other columns
df_covid = df_covid_removed_columns[columns_we_want_to_keep]

In [ ]:
# Check if the columns were removed
df_covid.info()

We then load another dataset so we can add data about the Human Development Index (HDI) for each country. The HDI is a composite index of life expectancy, education, and per capita income indicators, which are used to rank countries into four tiers of human development. This additional information will help us better understand the relationship between COVID-19 and various socioeconomic factors.

In [ ]:
# We load the new dataset
hdi = pd.read_csv('Data/human-development-index.csv')

Because we can then add a column with the HDI data for 2021 matching the countries in the covid dataset, because we only need data from the last year.

In [ ]:
# Filter HDI for 2021 only
hdi_2021 = hdi[hdi['Year'] == 2021]

# Merge using 'location' from df_covid and 'Entity' from hdi
df_merged = df_covid.merge(
    hdi_2021[['Entity', 'Human Development Index']], 
    left_on='location', 
    right_on='Entity', 
    how='left'
)

# Drop the extra 'Entity' column after merge, since we don't need it
df_merged = df_merged.drop(columns=['Entity'])

# Rename the column in the merged dataframe
df_merged = df_merged.rename(columns={'Human Development Index': 'human_development_index'})

In [ ]:
# Check how the dataset look and how we should proceed
df_merged

In [ ]:
# Shape of the dataframe after some cleaning
print(f"COVID dataframe shape after removing both some columns and rows: {df_merged.shape}")

We are isolating the remaining rows in the df_covid DataFrame to ensure it contains only country-level data. This allows us to clean the dataset and retain only the features that are most relevant for our analysis.

In [ ]:
# Since we seperated the OWID continent fields into it's own dataframe earlier, we now have to remove them again for the df_covid dataframe.
rows_to_remove = ["OWID_AFR", "OWID_ASI", "OWID_EUR", "OWID_EUN", "OWID_NAM", "OWID_OCE", "OWID_SAM"]
df_covid_removed_rows = df_merged[~df_merged['iso_code'].isin(rows_to_remove)]
df_covid_cleaned = df_covid_removed_rows.dropna(subset=['iso_code'])
df_covid_cleaned = df_covid_cleaned.drop(columns=['iso_code'])
df_covid_cleaned        

In [ ]:
# Check whether the rows were removed
print(f"COVID dataframe shape after removing rows: {df_covid_cleaned.shape}")
print(f"Removed {df_merged.shape[0] - df_covid_cleaned.shape[0]} rows from the dataframe.")
print(f"Removed {df_merged.shape[1] - df_covid_cleaned.shape[1]} column from the dataframe.")

In [ ]:
# Check for missing values in the DataFrame
df_covid_cleaned.isnull().sum()

We can see a lot of missing values for the human_development_index column, so we will impute them with the HDI from the sovereign countries they belong too. This step is important because missing values can lead to biased results or errors in our models.

In [ ]:
# Check which locations have missing HDI values
missing_hdi_locations = df_covid_cleaned[df_covid_cleaned['human_development_index'].isna()]
print(missing_hdi_locations['location'].unique())

In [ ]:
territory_to_country = {
    'American Samoa': 'United States',
    'Anguilla': 'United Kingdom',
    'Aruba': 'Netherlands',
    'Bermuda': 'United Kingdom',
    'Bonaire Sint Eustatius and Saba': 'Netherlands',
    'British Virgin Islands': 'United Kingdom',
    'Cayman Islands': 'United Kingdom',
    'Cook Islands': 'New Zealand',
    'Curacao': 'Netherlands',
    'Falkland Islands': 'United Kingdom',
    'Faroe Islands': 'Denmark',
    'French Guiana': 'France',
    'French Polynesia': 'France',
    'Gibraltar': 'United Kingdom',
    'Greenland': 'Denmark',
    'Guadeloupe': 'France',
    'Guam': 'United States',
    'Guernsey': 'United Kingdom',
    'Isle of Man': 'United Kingdom',
    'Jersey': 'United Kingdom',
    'Kosovo': 'Serbia',  # or leave as is if Kosovo has its own HDI
    'Martinique': 'France',
    'Mayotte': 'France',
    'Monaco': 'France',
    'Montserrat': 'United Kingdom',
    'Nauru': 'Nauru',
    'New Caledonia': 'France',
    'Niue': 'New Zealand',
    'North Korea': 'North Korea',
    'Northern Mariana Islands': 'United States',
    'Pitcairn': 'United Kingdom',
    'Puerto Rico': 'United States',
    'Reunion': 'France',
    'Saint Barthelemy': 'France',
    'Saint Helena': 'United Kingdom',
    'Saint Martin (French part)': 'France',
    'Saint Pierre and Miquelon': 'France',
    'Sint Maarten (Dutch part)': 'Netherlands',
    'Somalia': 'Somalia',
    'Tokelau': 'New Zealand',
    'Turks and Caicos Islands': 'United Kingdom',
    'United States Virgin Islands': 'United States',
    'Vatican': 'Italy',
    'Wallis and Futuna': 'France'
}

In [ ]:
# Map the territory to its sovereign country:
df_covid_cleaned['hdi_source_country'] = df_covid_cleaned['location'].map(territory_to_country)

In [ ]:
# Create a lookup for HDI values of sovereign countries
hdi_lookup = df_covid_cleaned.set_index('location')['human_development_index'].to_dict()

In [ ]:
# Fill missing HDI values with the sovereign country’s HDI
df_covid_cleaned['human_development_index'] = df_covid_cleaned.apply(
    lambda row: hdi_lookup.get(row['hdi_source_country'], row['human_development_index']) 
    if pd.isna(row['human_development_index']) else row['human_development_index'],
    axis=1
)

df_covid_cleaned.drop(columns=['hdi_source_country'], inplace=True)

In [ ]:
# Check for missing values in the DataFrame
df_covid_cleaned.isnull().sum()

In [ ]:
# Check which locations have missing HDI values
missing_hdi_locations = df_covid_cleaned[df_covid_cleaned['human_development_index'].isna()]
print(missing_hdi_locations['location'].unique())

We have found data for the human_development_index for 2021 for Nauru and Somalia on the https://hdr.undp.org/ website. We will use this data to fill in the missing values for these two countries in our dataset. We weren't able to find data on the human_development_index for North Korea, so we will impute it with the median value of the HDI for the other countries in the dataset. 

We will also impute the missing values for the columns total_cases, total_deaths, total_cases_per_million, total_deaths_per_million and life_expectancy with the median value.

In [ ]:
# Replace missing HDI values for Nauru and Somalia and impute North Korea with median value
replace_cell(df_covid_cleaned, df_covid_cleaned['location'] == 'Nauru', 'human_development_index', 0.692)
replace_cell(df_covid_cleaned, df_covid_cleaned['location'] == 'Somalia', 'human_development_index', 0.385)
fill_na_with_median(df_covid_cleaned, "human_development_index")
fill_na_with_median(df_covid_cleaned, "total_cases")
fill_na_with_median(df_covid_cleaned, "total_deaths")
fill_na_with_median(df_covid_cleaned, "total_cases_per_million")
fill_na_with_median(df_covid_cleaned, "total_deaths_per_million")
fill_na_with_median(df_covid_cleaned, "life_expectancy")
df_covid_cleaned


In [ ]:
# Check for missing values in the DataFrame
df_covid_cleaned.isnull().sum()

In [ ]:
# Check for duplicates in the DataFrame
df_covid_cleaned.duplicated().sum()

---

### 4. Hypotese 1: Higher population size is associated with higher total COVID-19 deaths, but not necessarily with higher deaths per capita. 


 #### 4.1 Explore

 #### 4.2 Data Modelling

---

### 5. Hypotese 2: Countries with a higher Human Development Index (HDI) have experienced lower COVID-19 death rates per capita

 #### 5.1 Explore

 #### 5.2 Data Modelling

---

### 6. Hypotese 3: Countries with a higher life expectancy and older populations (e.g. higher median age, % aged 65+, etc.) have experienced higher COVID-19 death rates.

 #### 6.1 Explore

The dependent variable is `total_deaths_per_million`, which represents the total number of COVID-19 deaths per million people in each country. This variable is crucial for understanding the impact of the pandemic on different populations and will be used to assess the relationship with independent variables such as `median_age`, `aged_65_older`, `aged_70_older` and `life_expectancy`. 

It's important to mention, not all countries are represented in the dataset, since the countries with missing data on these variables were removed. This means that the analysis will only include countries for which we have complete data on these variables.

***6.1.1 Descriptive Statistics***

First we look at some of the descriptive statistics of the `df_age_cleaned` DataFrame to get an overview of the data. This includes the mean, median, standard deviation, and other statistics for each column.

In [ ]:
# Gives summary statistics for all numerical columns in the dataset
df_age_cleaned.describe()

The data shows large variation across countries in both age-related factors and COVID-19 death rates. Median age ranges from 15 to 48 years, and life expectancy from 53 to nearly 85, indicating diverse population structures. COVID-19 deaths per million also vary widely, from 0 to over 6,600, with a high standard deviation—suggesting age and life expectancy could meaningfully relate to differences in death rates.

***6.1.2 Normality***

To tests whether numeric columns follow a normal distribution, we can use the D'Agostino and Jarque-Bera tests. These tests are designed to assess the skewness and kurtosis of the data, which are key indicators of normality.

In [ ]:
# Function to test normality of numeric columns
def check_normality(df):
    num_cols = [col for col in df.select_dtypes(include=['float64', 'int64']).columns if col not in ['location', 'continent']]
    
    rows = []

    for col in num_cols:
        data = df[col]
        skewness = data.skew()
        kurtosis = data.kurt()
        dagostino = stats.normaltest(data)
        jb = stats.jarque_bera(data)

        normal = "No"
        if dagostino.pvalue > 0.05 and jb.pvalue > 0.05 and abs(skewness) < 1:
            normal = "Yes"
        elif dagostino.pvalue > 0.01 and abs(skewness) < 2:
            normal = "Partial"

        rows.append({
            'Column': col,
            'Skewness': round(skewness, 3),
            'Kurtosis': round(kurtosis, 3),
            "D'Agostino p-value": f"{dagostino.pvalue:.2e}",
            "Jarque-Bera p-value": f"{jb.pvalue:.2e}",
            'Normally Distributed?': normal
        })

    return pd.DataFrame(rows)

# Run normality checks on all numeric columns
check_normality(df_age_cleaned)

The normality tests show that none of the key variables follow a normal distribution. All columns—total_deaths_per_million, median_age, aged_65_older, aged_70_older, and life_expectancy—exhibit significant skewness and/or kurtosis, with very low p-values from both the D'Agostino and Jarque-Bera tests, confirming deviations from normality.

For visualization purposes, we want to see how the data looks like in histograms. This will help us understand the distribution of the data and identify any potential outliers or skewness.

In [ ]:
def visualize_selected_histograms(df):
    """
    Visualizes the distribution of selected numeric columns from df_age_cleaned with histograms.
    """
    selected_cols = [
        'total_deaths_per_million',
        'life_expectancy',
        'median_age',
        'aged_65_older',
        'aged_70_older'
    ]

    n = len(selected_cols)
    n_cols = 3
    n_rows = (n + n_cols - 1) // n_cols 

    fig, axes = plt.subplots(n_rows, n_cols, figsize=(15, 4 * n_rows))
    axes = axes.flatten()

    for i, col in enumerate(selected_cols):
        sns.histplot(df[col], kde=True, ax=axes[i])
        axes[i].set_title(f'Distribution of {col}')
        axes[i].set_xlabel(col.replace('_', ' ').title())

    # Hide unused axes if any
    for j in range(i + 1, len(axes)):
        fig.delaxes(axes[j])

    plt.tight_layout()
    plt.show()

In [ ]:
visualize_selected_histograms(df_age_cleaned)

Based on the statistical tests and visualizations, none of the numeric variables appear to be normally distributed. The distributions show that total deaths per million and the age-related variables (especially % aged 65+ and 70+) are right-skewed, meaning most countries have lower values but a few have very high ones. In contrast, life expectancy is more normally distributed, and median age is fairly spread out across countries. 

***6.1.3 Outliers***

To identify outliers in the `df_age_cleaned` DataFrame, we can use the Interquartile Range (IQR) method. This involves calculating the first (Q1) and third quartiles (Q3) for each numeric column, then determining the IQR as Q3 - Q1. Outliers are defined as values that fall below Q1 - 1.5 * IQR or above Q3 + 1.5 * IQR.

In [ ]:
# Loop through selected columns
for column in ['life_expectancy', 'median_age', 'aged_65_older', 'aged_70_older']:
    # Calculate Q1 (25th percentile) and Q3 (75th percentile)
    Q1 = df_age_cleaned[column].quantile(0.25)
    Q3 = df_age_cleaned[column].quantile(0.75)
    IQR = Q3 - Q1  # Interquartile Range

    # Define the lower and upper bounds for detecting outliers
    lower_bound = Q1 - 1.5 * IQR
    upper_bound = Q3 + 1.5 * IQR

    # Find rows where the value is outside the normal range
    outliers = df_age_cleaned[
        (df_age_cleaned[column] < lower_bound) | 
        (df_age_cleaned[column] > upper_bound)
    ]

    # Print the number of outliers found for the column
    print(f"  {column}: {len(outliers)} outliers detected")

There are no outliers in most variables, except for aged_70_older, which has 1 outlier. This indicates that the data is generally consistent, with only one unusually high or low value in the aged_70_older group. We will keep this outlier, as our dataset is small and it may represent a country with unique characteristics that could be important for our analysis.

To explore how age-related factors impact COVID-19 deaths, we group `life_expectancy`, `aged 65+`, and `aged 70+` into “low” and “high” categories in order to compare death rates across these groups. We define "low" and "high" based on the median values of each variable, which allows us to categorize countries into two groups for analysis.

In [ ]:
def boxplot_by_age_factors(df):
    """
    Categorizes life_expectancy, aged_65_older, and aged_70_older into
    'Low' and 'High' groups and plots boxplots of total_deaths_per_million for each.
    """
    # Define the factors to categorize
    factors = ['life_expectancy', 'aged_65_older', 'aged_70_older']
    
    # Categorize into 'Low' and 'High' using median split
    for col in factors:
        median_val = df[col].median()
        df[f'{col}_group'] = df[col].apply(lambda x: 'Low' if x < median_val else 'High')

    # Set up subplots
    fig, axes = plt.subplots(1, 3, figsize=(18, 5))
    
    for i, col in enumerate(factors):
        sns.boxplot(
            x=f'{col}_group',
            y='total_deaths_per_million',
            data=df,
            ax=axes[i],
            palette='Set2'
        )
        axes[i].set_title(f'Deaths per Million by {col.replace("_", " ").title()} Grouped')
        axes[i].set_xlabel('')
        axes[i].set_ylabel('Deaths per Million')
    
    plt.tight_layout()
    plt.show()

In [ ]:
boxplot_by_age_factors(df_age_cleaned)      

# Drop the age group columns after visualization
df_age_cleaned = df_age_cleaned.drop(columns=['life_expectancy_group', 'aged_65_older_group', 'aged_70_older_group']) 

We see that some countries with younger populations or lower life expectancy still experienced high death rates. These outliers suggest that other factors that are not represented in the data (e.g., healthcare quality, policy respons, etc) may also play a significant role.

But overall the boxplots show a clear relationship between the age-related variables and total deaths per million. Countries with larger percentages of older populations (aged 65+ and 70+) and longer life expectancies tend to have higher total deaths per million. This suggests that age and life expectancy are important factors in understanding COVID-19 death rates.

***6.1.4 Correlation***

To assess the relationship between age-related factors and COVID-19 death rates, we will use a Heatmap to visualize the correlation matrix of the numeric variables in the `df_age_cleaned` DataFrame. This will help us identify any strong correlations between the variables, particularly between age-related factors and total deaths per million.

In [ ]:
# Function creates a correlation matrix from our DataFrame. 
def my_corr(df):
    cormat = df.drop(columns=['continent', 'location']).corr() #checks how strongly each pair of columns are related and drops column 'wine-type'. 
    return cormat

# function that takes the correlation matrix and draws a heatmap (using Seaborn)
def my_corr_plot(cormat):
    sns.heatmap(cormat, cmap = 'viridis',  annot=True, fmt=".2f", square=True, linewidths=.2) #cmap - sets the color style. annot=true - means the numbers will be shown on the heatmap. 
    plt.show()


my_corr_plot(my_corr(df_age_cleaned))

The HeatMap shows a moderate to strong positive correlation between age-related factors and COVID-19 death rates. Median age, percentage aged 65+, and aged 70+ all correlate strongly (~0.66–0.67) with deaths per million, while life expectancy has a moderate correlation (0.52). Median age and life expectancy themselves are highly correlated (0.83), reflecting that countries with older populations tend to have longer life expectancy.

The strong correlation between aged 65+ and aged 70+ is expected since these groups overlap (70+ a subset of 65+), indicating multicollinearity that should be considered in further analysis.

***6.1.5 Scatter Plots***

We will create scatter plots to visualize the relationships between total deaths per million and the age-related factors: median age, aged 65+ and life expectancy. This will help us understand how these variables relate to COVID-19 death rates.

In [ ]:
# Visualise the features and the response using scatterplots
sns.pairplot(df_age_cleaned, x_vars=['life_expectancy', 'median_age', 'aged_65_older'], y_vars='total_deaths_per_million', height=5, aspect=1)

The scatter plots show a positive relationship between total deaths per million and the age-related factors. Especially countries with higher median ages and larger percentages of people aged 65+ have a clear postive relationship and tend to have higher total deaths per million. Life expectancy also shows a positive relationship, but with more variability.

 #### 6.2 Data Modelling

Now we want to train a model to predict total deaths per million based on the age-related factors. We will use a linear regression model for this purpose, as it is a simple yet effective way to understand the relationship between the dependent variable (total deaths per million) and independent variables (median age, aged 65+, aged 70+, and life expectancy).

***6.2.1 Multiple Linear Regression***

To assess the relationship between total deaths per million and the age-related factors, we will use a multiple linear regression model. This model will allow us to quantify how each independent variable (median age, aged 65+ and life expectancy) contributes to the dependent variable (total deaths per million).

In [ ]:
# Create a Python list of feature names
feature_cols = ['life_expectancy', 'median_age', 'aged_65_older']

# Use the list to select a subset of the original DataFrame
X = df_age_cleaned[feature_cols]

# Print the first 5 rows
X.head()

In [ ]:
# Select a Series from the DataFrame for y
y = df_age_cleaned['total_deaths_per_million']

# Print the first 5 values
y.head()

In [ ]:
# Check the type and shape of X
print(f"Type of X: {type(X)}")
print(f"Shape of X: {X.shape}")

# Check the type and shape of y
print(f"Type of y: {type(y)}")
print(f"Shape of y: {y.shape}")

6.2.1.1 - Now we are going to split X and y variables into training and testing sets. This is important to evaluate the model's performance on unseen data and avoid overfitting.

In [ ]:
# Split the dataset into training and testing sets
X_train, X_test, y_train, y_test = train_test_split(X, y, random_state=1)

# Default split 75:25
print(f"Shape of X train: {X_train.shape} and y train: {y_train.shape}")
print(f"Shape of X test: {X_test.shape} and y test: {y_test.shape}")

In [ ]:
# Print the first 5 values
print(f"First 5 rows of X train:\n{X_test.head()}")
print(f"First 5 values of y train:\n{y_test.head()}")

6.2.1.2 - We can now create a ultiple linear regression model using the training data.

In [ ]:
# Create a model
linreg = LinearRegression()

# Fit the model to our training data
linreg.fit(X_train, y_train)

In [ ]:
# The intercept and coefficients of the model
print('b0 =', linreg.intercept_)
print('bi =', linreg.coef_)

In [ ]:
# Pair the feature names with the coefficients
list(zip(feature_cols, linreg.coef_))

Looking and the slopes (coefficients) for life expectancy, median age and aged 65+, we see that: 

- For each additional year of life expectancy, predicted deaths per million decrease by ~10.6.
- For each additional year of median age, predicted deaths per million increase by ~76.3
- For each 1% increase in population aged 65+, deaths per million increase by ~54.2.

Which indicates that older populations correlate with higher COVID-19 death rates, while higher life expectancy may slightly reduce it.

6.2.1.3 - Now we test the the model with the test data to evaluate its performance. We will use the R-squared value to assess how well the model explains the variance in total deaths per million.

In [ ]:
# Make predictions on the testing set
y_predicted = linreg.predict(X_test)

y_predicted

In [ ]:
# Mean Absolute Error (MAE) is the mean of the absolute value of the errors:
print(f"Mean Absolute Error (MAE): {metrics.mean_absolute_error(y_test, y_predicted)}")

# Mean Squared Error (MSE) is the mean of the squared errors
print(f"Mean Squared Error (MSE): {metrics.mean_squared_error(y_test, y_predicted)}")

# Root Mean Squared Error (RMSE) is the square root of the mean of the squared errors
print(f"Root Mean Squared Error (RMSE): {np.sqrt(metrics.mean_squared_error(y_test, y_predicted))}")

In [ ]:
# R-squared
print(f"Explained variance score: {r2_score(y_test, y_predicted)}")

In [ ]:
# Visualise the regression results
plt.title('Multiple Linear Regression')
plt.scatter(y_test, y_predicted, color='blue')
plt.show()

The model captures a meaningful relationship between age-related factors and COVID-19 death rates, explaining over half the variance in mortality across countries. The positive coefficients for median age and aged 65+ confirm that older populations generally experience higher death rates, while higher life expectancy seems protective or associated with lower deaths.

However, the prediction errors (MAE and RMSE) indicate moderate inaccuracies, so other factors beyond age-related variables likely influence COVID-19 deaths as well. This suggests the model is useful for understanding broad trends but may have limitations in precise predictions due to unaccounted variables or data variability.

***6.2.2 Decision Tree***

To further explore the relationship between age-related factors and COVID-19 death rates, we will use a Decision Tree model. This model will allow us to capture non-linear relationships and interactions between the independent variables and the dependent variable.

But first we got to create categories `total_deaths_per_million` in order to use classification algorithms.

In [ ]:
# Divide the continuous variable into 3 categories using quantiles
df_age_cleaned['death_rate_category'] = pd.qcut(df_age_cleaned['total_deaths_per_million'], q=3, labels=['Low', 'Medium', 'High'])

In [ ]:
# Define the feature columns
feature_cols = ['life_expectancy', 'median_age', 'aged_65_older']

# Select only those columns plus the target column (total_deaths_per_million)
selected_cols = feature_cols + ['death_rate_category']

# Extract the subset DataFrame
df_subset = df_age_cleaned[selected_cols]

# Convert to numpy array
array = df_subset.to_numpy()

# Create two (sub) arrays from it: 
# X - features, all rows, all columns but the last one and y - labels, all rows, the last column
X, y = array[:, :-1], array[:, -1]

# Separate input data into classes based on labels of diagnoses
class0 = np.array(X[y==0])
class1 = np.array(X[y==1])
class2 = np.array(X[y==2])

6.2.2.1 - We will split the data into training and testing sets, just like we did for the multiple linear regression model. This is important to evaluate the model's performance on unseen data and avoid overfitting.

In [ ]:
# Split the dataset into into training and testing sets in proportion 8:2 - 80% of it as training data and 20% as a validation dataset
X_train, X_test, y_train, y_test = model_selection.train_test_split(X, y, test_size=0.15, random_state=12)

# Build Decision Trees Classifier 
classifier = DecisionTreeClassifier(max_depth = 5)
classifier.fit(X_train, y_train)

In [ ]:
# Install the graphviz package for DT visualisation
%pip install graphviz
import graphviz

# Draw tree from the trained data by graphviz package
dot_data = tree.export_graphviz(classifier, out_file=None, 
                         feature_names=feature_cols, class_names = True,        
                         filled=True, rounded=True, proportion = False,
                         special_characters=True) 

# Result DT saved in file age.pdf
graph = graphviz.Source(dot_data)
graph.render("Data/age") 

# Show the graph
graph 

test

---

### 7. Countries or continents with higher vaccination rates experienced lower COVID-19 death rates.

 #### 7.1 Explore

 #### 7.2 Data Modelling